In [1]:
! pip install transformers sentencepiece

In [2]:
!git lfs install
!git clone https://huggingface.co/cointegrated/rubert-tiny2

Git LFS initialized.
Cloning into 'rubert-tiny2'...
remote: Enumerating objects: 59, done.
remote: Total 59 (delta 0), reused 0 (delta 0), pack-reused 59 (from 1)
Unpacking objects: 100% (59/59), 985.64 KiB | 7.88 MiB/s, done.
Filtering content: 100% (3/3), 225.10 MiB | 123.53 MiB/s, done.


In [3]:
import random
import numpy as np
import pandas as pd
import torch
from torch import nn
from collections import OrderedDict
from transformers import AutoTokenizer, AutoModel
from torch.utils.tensorboard import SummaryWriter

In [4]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed = 42
set_seed(seed)

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
        self.base = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

        # freezing base model weights
        for param in self.base.parameters():
            param.requires_grad = False

        n_dim = 312
        self.head = nn.Sequential(OrderedDict( [('dropout', torch.nn.Dropout(.2)),
                                                ('fc_1' , nn.Linear(n_dim, n_dim//2)),
                                                ('relu_1' , nn.ReLU()),
                                                ('batchnorm_1' , nn.BatchNorm1d(n_dim//2, eps=1e-12)),
                                                ('dropout', torch.nn.Dropout(.2)),
                                                ('fc_3' , nn.Linear(n_dim//2, 2, bias=False))
                    ]))

    def forward(self, tokens):
        model_output = self.base(**tokens)
        result = self.head(model_output.pooler_output)
        return result
    
    def get_loss(self, texts, labels):
        targets = labels.long().to(device)  # Convert labels to long integers
        tokens = self.tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to(device)
        outputs = self.forward(tokens)
        return criterion(outputs, targets)
    
    def eval_loss(self, dataloader):
        batch_indx = np.random.randint(len(dataloader)+1, size=batch_size)
        batch_texts = [dataloader.dataset[i][0] for i in batch_indx]
        batch_labels = torch.Tensor([train_dataloader.dataset[i][1] for i in batch_indx])
        return self.get_loss(batch_texts, batch_labels).item()

In [7]:
# model_path = './CrossEncoderModel'
model = Model().to(device)

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

In [11]:
# Unpickle dataset
df = pd.read_pickle('/kaggle/input/dataset-qa-last/data_last.pkl')
df.head()

,query,text,label,clean_text,clean_query,embedding_text,embedding_query
0,Когда был спущен на воду первый миноносец «Спо...,Зачислен в списки ВМФ СССР 19 августа 1952 год...,1,Зачислен в списки ВМФ СССР 19 августа 1952 год...,Когда был спущен на воду первый миноносец «Спо...,"[-0.0030975677, -0.018142669, -0.0058722952, 0...","[0.00064380514, 0.0074218363, -0.03353223, -0...."
1,Как долго существовало британское телевизионно...,"Хрустальный лабиринт (""The Crystal Maze"") — бр...",1,"Хрустальный лабиринт (""The Crystal Maze"") — бр...",Как долго существовало британское телевизионно...,"[0.0026477594, -0.026646728, 0.0009579654, -0....","[-0.029795218, -0.01173853, -0.00032150946, -0..."
2,Когда родилась Князева Марина Леонидовна?,Князева Марина Леонидовна (род. 7 мая 1952 г.)...,1,Князева Марина Леонидовна (род. 7 мая 1952 г.)...,Когда родилась Князева Марина Леонидовна?,"[-0.036747612, -0.012604811, -0.0109199695, -0...","[-0.036575466, -0.010551005, -0.04117768, -0.0..."
3,Кто был главным художником мира Зен?,"В книге ""Half-Life 2: Raising the Bar"" художни...",1,"В книге ""Half-Life 2: Raising the Bar"" художни...",Кто был главным художником мира Зен?,"[-0.02514373, -0.023727695, -0.04738828, 0.011...","[-0.022960061, -0.013048667, -0.018877652, -0...."
4,Как звали предполагаемого убийцу Джона Кеннеди?,В 1966 году окружной прокурор Нового Орлеана Д...,1,В 1966 году окружной прокурор Нового Орлеана Д...,Как звали предполагаемого убийцу Джона Кеннеди?,"[0.0074619167, -0.024880972, -0.026498705, 0.0...","[-0.0124044325, -0.0020067864, -0.030558525, 0..."


In [12]:
# Train-test split

from sklearn.model_selection import train_test_split

df_filtered = df[['clean_query', 'clean_text', 'label']]

# Группируем по запросам
grouped = df_filtered.groupby('clean_query').agg({'clean_text': list, 'label': list}).reset_index()

train, test_val = train_test_split(grouped, test_size=0.2, random_state=42)

# Разворачиваем списки текстов и меток обратно в строки для каждой подвыборки
train = train.explode(['clean_text', 'label']).reset_index(drop=True)
test_val = test_val.explode(['clean_text', 'label']).reset_index(drop=True)

# Проверка результата
print("Train size:", len(train))
print("Test/Validation size:", len(test_val))

Train size: 37562
Test/Validation size: 9447


In [13]:
train.head()

,clean_query,clean_text,label
0,Сколько официальных языков в США?,Языки Соединённых Штатов Америки — множество я...,1
1,Сколько официальных языков в США?,Хотя Соединённые Штаты не имеют на федеральном...,1
2,Сколько официальных языков в США?,Самый распространённый в США язык — английский...,0
3,Сколько официальных языков в США?,Официальным языком образования и делопроизводс...,0
4,Сколько официальных языков в США?,В США в штате Нью-Йорк в 2009 году внесена поп...,0


In [16]:
X_train, y_train  = (train['clean_query']+' [SEP] '+train['clean_text']).to_numpy(), train['label'].to_numpy()
X_test, y_test = (test_val['clean_query']+' [SEP] '+test_val['clean_text']).to_numpy(), test_val['label'].to_numpy()

In [17]:
from torch.utils.data import Dataset

class PandasDataset(Dataset):
    def __init__(self, df):
        self.dataframe = df.reset_index()

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        return list(self.dataframe.iloc[index])[1:]

In [19]:
# Hyperparameters
num_epochs = 8
batch_size = 64
learning_rate = 1e-4
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.008)

In [20]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(PandasDataset(pd.DataFrame([X_train, y_train]).T),
                              batch_size=batch_size)
test_dataloader = DataLoader(PandasDataset(pd.DataFrame([X_test, y_test]).T),
                              batch_size=batch_size)

In [21]:
writer = SummaryWriter()

In [22]:
import time

# Training loop
total_step = len(train_dataloader)
best_val_loss = float('inf')
for epoch in range(num_epochs):
    model.train()
    epoch_start_time = time.time()  # Start timing the epoch
    train_loss = 0.0
    for texts, labels in train_dataloader:
        loss = model.get_loss(texts, labels)
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()
        train_loss += loss.item() * len(texts)  # Accumulate loss

    train_loss /= len(train_dataloader.dataset)  # Compute average loss
    
    writer.add_scalar("Loss/train", train_loss, epoch)

    # Evaluate loss after each epoch
    model.eval()
    with torch.no_grad():
        test_loss = sum(model.get_loss(batch[0], batch[1]).item() * len(batch[0]) for batch in test_dataloader) / len(test_dataloader.dataset)

    if test_loss < best_val_loss:
        best_val_loss = test_loss
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch,
            'loss': loss
        }, 'best_model.ckpt')
        print('\nsave_model\n')

    epoch_duration = time.time() - epoch_start_time  # Calculate epoch duration
    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Time: {epoch_duration:.2f} seconds')


save_model

Epoch [1/8], Train Loss: 0.6652, Test Loss: 0.5925, Time: 121.06 seconds

save_model

Epoch [2/8], Train Loss: 0.6037, Test Loss: 0.5763, Time: 119.97 seconds
Epoch [3/8], Train Loss: 0.5814, Test Loss: 0.5794, Time: 119.22 seconds
Epoch [4/8], Train Loss: 0.5763, Test Loss: 0.5809, Time: 119.20 seconds
Epoch [5/8], Train Loss: 0.5748, Test Loss: 0.5821, Time: 119.26 seconds
Epoch [6/8], Train Loss: 0.5717, Test Loss: 0.5798, Time: 119.30 seconds
Epoch [7/8], Train Loss: 0.5710, Test Loss: 0.5784, Time: 119.10 seconds
Epoch [8/8], Train Loss: 0.5682, Test Loss: 0.5795, Time: 119.20 seconds


In [23]:
writer.close()

In [26]:
model2 = Model().to(device)

checkpoint = torch.load('./best_model.ckpt', weights_only=True)

model2.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

# Если вы планируете использовать модель для обучения
#model.train()

# Или если планируете использовать только для инференса
model2.eval()

Model(
  (base): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=T

In [27]:
def testing(model_f):
    correct, total = 0, 0
    with torch.no_grad():
        for texts, labels in test_dataloader:
            labels = labels.to(device).long() # Crucial: Convert labels to long
            tokens = model_f.tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to(device)
            outputs = model_f(tokens)
            _, predicted = torch.max(outputs, 1) # Correct way to get predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy: {100 * correct / total:.2f}%')

In [28]:
testing(model)

Accuracy: 72.28%


In [29]:
testing(model2)

Accuracy: 72.05%


### Ranking

In [30]:
def custom_metric(df_sorted):
    n = len(df_sorted)
    k = df_sorted['label'].sum()
    if k == 0:
        return 1
    top_k = max(3, int(k))
    
    top_k_answers = df_sorted.iloc[:top_k]
    
    correct_in_top_k = top_k_answers['label'].sum()
    
    score = correct_in_top_k / k
    return score


In [31]:
def rank_answers_by_relevance(df, model_f):
    """
    Rank answers based on relevance to a query using a fine-tuned model.
    :param df: DataFrame with 'query' and 'text' columns
    :param model_f: The fine-tuned model instance
    :return: DataFrame with additional 'relevance' column, sorted by relevance
    """
    relevance_scores = []
    
    for _, row in df.iterrows():
        query = row['clean_query']
        answer = row['clean_text']
        
        # Tokenize using the model's tokenizer
        inputs = model_f.tokenizer(
            query,
            answer,
            add_special_tokens=True,
            return_tensors='pt',
            max_length=512,
            truncation=True,
            padding='max_length'
        ).to(device)

        # Get model prediction and calculate relevance score
        with torch.no_grad():
            outputs = model_f(inputs)
            relevance_score = torch.softmax(outputs, dim=1)[0][1].item()  # Get probability of relevance class
            relevance_scores.append(relevance_score)
    
    # Add relevance scores to DataFrame and sort
    df['relevance'] = relevance_scores
    df_sorted = df.sort_values(by='relevance', ascending=False).reset_index(drop=True)
    
    return df_sorted


In [32]:

res = []
for query, text in df.groupby('clean_query'):
    ranked_df = rank_answers_by_relevance(text, model)
    score = custom_metric(ranked_df)
    n = len(ranked_df)
    k = ranked_df['label'].sum()
    res.append([query, score, n, k])

summ_score = sum([elem[1] for elem in res]) / len(res)
print("Average Score:", summ_score)


Average Score: 0.6350554752536202


In [33]:
test_val

,clean_query,clean_text,label
0,Где родился Митрополит Вениамин?,Митрополит Вениамин (в миру Иван Афанасьевич Ф...,1
1,Где родился Митрополит Вениамин?,"Митрополит Вениамин (, в миру Василий Костаки;...",1
2,Где родился Митрополит Вениамин?,Митрополит Вениамин (в миру Борис Николаевич П...,1
3,Где родился Митрополит Вениамин?,Вениамин (в миру Василий Антонович Муратовский...,0
4,Где родился Митрополит Вениамин?,Архиепископ Вениамин (в миру Вениамин Михайлов...,0
...,...,...,...
9442,Сколько стоит проезд на метро в Москве за одну...,В конце апреля 2018 года Московский метрополит...,0
9443,Сколько стоит проезд на метро в Москве за одну...,С 2004 по 2011 год стоимость проезда на метро ...,0
9444,Сколько стоит проезд на метро в Москве за одну...,"2 апреля 2013 полностью обновлены тарифы, введ...",0
9445,Когда был сформирован 130-й Ордена Суворова Ла...,Корпус был сформирован 5 июня 1944 года приказ...,1


In [34]:

res = []
for query, text in test_val.groupby('clean_query'):
    ranked_df = rank_answers_by_relevance(text, model2)
    score = custom_metric(ranked_df)
    n = len(ranked_df)
    k = ranked_df['label'].sum()
    res.append([query, score, n, k])

summ_score = sum([elem[1] for elem in res]) / len(res)
print("Average Score:", summ_score)

Average Score: 0.6346720335126745


## Дообучение после разморозки

In [ ]:
for param in model2.base.parameters():  # СМОТРИ КАКАЯ МОДЕЛЬ model если не загружал, model2 если загружал
    param.requires_grad = True

# Reinitialize the optimizer to include all parameters
# Reduce the learning rate for fine-tuning (suggested value: 1e-5)
fine_tuning_lr = 1e-4
optimizer = torch.optim.AdamW(model2.parameters(), lr=fine_tuning_lr, weight_decay=0.01)

In [ ]:
num_fine_tune_epochs = 12
total_step = len(train_dataloader)
best_val_loss = float('inf')

for epoch in range(num_fine_tune_epochs):
    model2.train()
    epoch_start_time = time.time()  # Start timing the epoch
    train_loss = 0.0
    for texts, labels in train_dataloader:
        loss = model2.get_loss(texts, labels)
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model2.parameters(), max_norm=1)
        optimizer.step()
        train_loss += loss.item() * len(texts)  # Accumulate loss

    train_loss /= len(train_dataloader.dataset)  # Compute average loss

    # Evaluate loss after each epoch
    model2.eval()
    with torch.no_grad():
        test_loss = sum(model2.get_loss(batch[0], batch[1]).item() * len(batch[0]) for batch in test_dataloader) / len(test_dataloader.dataset)

    if test_loss < best_val_loss:
        best_val_loss = test_loss
        torch.save({
            'model_state_dict': model2.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch,
            'loss': loss
        }, 'best_model_fine_tuned.ckpt')
        print('\nsave_model\n')

    epoch_duration = time.time() - epoch_start_time  # Calculate epoch duration
    print(f'Epoch [{epoch + 1}/{num_fine_tune_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Time: {epoch_duration:.2f} seconds')


In [ ]:
testing(model2)

In [ ]:
model3 = Model().to(device)

checkpoint = torch.load('best_model_fine_tuned.ckpt', weights_only=True)

model3.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model3.eval()

testing(model3)